In [1]:
# IMPORTING UPROOT
import uproot

# IMPORTING SUPPORT PACKAGES
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# IMPORTING ADDITIONAL PACKAGES
from ipywidgets import interact, widgets
import sys
import glob, re
import os
import time

# IMPORTING SUPPORT METHODS
import PLOTTER
import importlib # Useful to reload packages

In [2]:
# Extracting tree names for a root file
file = uproot.open('../ROOT FILES/SIM_minerva_00110000_Subruns_0001_MasterAnaDev_Ana_Tuple_v22r1p1.root')
trees = file.keys()
trees

['Header;1', 'MasterAnaDev;1', 'Truth;1', 'Meta;1']

In [3]:
# Generating list of all ROOT files
filenames = sorted(glob.glob("../ROOT FILES/SIM*"))

In [4]:
# Generating required directories
mother_directory = 'UNPACKED_ROOT'
os.mkdir(mother_directory) # Creating new directory
for tree in trees:
    os.mkdir(mother_directory + '/' + tree)

In [5]:
trees = file.keys()
trees.insert(0, 'Select Tree')
drop_down1 = widgets.Dropdown(options=trees,
                                description='🌴 Tree:',
                                disabled=False,
                                layout={'width': 'max-content'})
drop_down1

Dropdown(description='🌴 Tree:', layout=Layout(width='max-content'), options=('Select Tree', 'Header;1', 'Maste…

In [6]:
# Search inside branches list (OPTIONAL)
search1 = widgets.Text(placeholder='Type something',
                    description='🔎 Search:',
                    disabled=False)
search1

Text(value='', description='🔎 Search:', placeholder='Type something')

In [7]:
# Let's see inside a tree
tree = file[drop_down1.value]
branches = tree.keys() 
branches.insert(0, 'Select Branch')
branch_list = []
#next((branch_list.append(item) for item in branches if search1.value in item), None)
[branch_list.append(item) for item in branches if item.count(search1.value) != 0]
options = ['No results'] if len(branch_list) == 0 else (branches if (search1.value == '') else branch_list)
drop_down2 = widgets.Dropdown(options=options,
                                description='🍂 Branch:',
                                disabled=False,
                                layout={'width': 'max-content'})
drop_down2

Dropdown(description='🍂 Branch:', layout=Layout(width='max-content'), options=('mc_Q2',), value='mc_Q2')

In [8]:
# Creating branch directory (if it doesn't exist)
branch = drop_down2.value
tree_name = drop_down1.value
try:
    os.mkdir(mother_directory + '/' + tree_name + '/' + branch)
except FileExistsError:
    pass

# Defining a method that outputs the .txt file for an specified root file
def exportTXT(filename):
    file1 = uproot.open(filename)
    tree = file1[tree_name]
    new_name = filename.split("FILES/")[1]
    file2 = mother_directory + '/' + tree_name + '/' + branch + '/' + new_name + '.txt'
    print(file2)
    data = tree.arrays()[branch].to_numpy()
    np.savetxt(file2, data, delimiter=',')

In [9]:
# Exporing each root file using parallel programming
start = time.time()
export = Parallel(n_jobs=-2)(delayed(exportTXT)(filename) for filename in filenames)
end = time.time()
print((end - start)/60, 'min')

135.66677713394165


***
#### <font color=orange>UNUSED CODE</font>
***

In [30]:
# Reading all files and branches 
# UPDATE: NOT EFFICIENT
i = 1
for filename in filenames:
    file = uproot.open(filename)
    for elm in trees:
        tree = file[elm]
        branches = tree.keys()
        for branch in branches:
            try: # If we have repeated branches, only the first one will be saved
                os.mkdir(mother_directory + '/' + elm + '/' + branch)
            except FileExistsError:
                pass
            file2 = mother_directory + '/' + elm + '/' + branch + '/MasterAnaDev_Ana_Tuple' + str(i) + '.txt'
            data = tree.arrays()[branch].to_numpy()
            np.savetxt(file2, data, delimiter=',')
    i += 1
    
# Exporting each root file data into a .txt file (for each root file)
# UPDATE: THIS WORKS, BUT EVEN 1 ROOT FILE TAKES TOO LONG, WILL TRY WITH PARALLEL PROGRAMMING
i = 1
for filename in filenames: 
    file = uproot.open(filename) # There is an error here, I forgot to update the tree.
    file2 = mother_directory + '/' + tree_name + '/' + branch + '/MasterAnaDev_Ana_Tuple' + str(i) + '.txt'
    data = tree.arrays()[branch].to_numpy()
    np.savetxt(file2, data, delimiter=',')
    i += 1

KeyboardInterrupt: 